In [1]:
import os

%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow'

## 1. Update `config.yaml`

```yaml
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_url: https://github.com/entbappy/Branching-tutorial/master/winequality-data.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion
```

## 4. Update the Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [4]:
from mlflow_project.constants import *
from mlflow_project.utils.common import read_yaml, create_directories

from mlflow_project import logger


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Creates the root directory and returns
        the configuration for data ingestion.

        Returns:
            DataIngestionConfig: Configuration for data ingestion.
        """
        data_ingestion = self.config.data_ingestion

        create_directories([data_ingestion.root_dir])

        file_path = os.path.join(data_ingestion.root_dir, ".gitkeep")
        if not os.path.exists(file_path):
            with open(file_path, "w") as f:
                logger.info(
                    f"Creating file: .gitkeep in directory {data_ingestion.root_dir}"
                )
                pass

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(data_ingestion.root_dir),
            source_url=str(data_ingestion.source_url),
            local_data_file=Path(data_ingestion.local_data_file),
            unzip_dir=Path(data_ingestion.unzip_dir),
        )

        return data_ingestion_config

## 6. Update the Components

In [5]:
import os
import zipfile
import urllib.request as request

from mlflow_project.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the zipped data file if it does not already exist."""
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file,
            )
            logger.info(f"{file_name} downloaded with the following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(self.config.local_data_file)}"
            )

    def extract_zip_file(self):
        """Extracts the contents from zipped file.

        Creates a directory if it does not already exist
        and extracts the contents fo the zipped file into the directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as f:
            f.extractall(unzip_path)

## 7. Update the Pipeline

In [6]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2023-10-24 17:53:26,155 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-10-24 17:53:26,158 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-10-24 17:53:26,159 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: schema.yaml
[ 2023-10-24 17:53:26,160 ] 53 common mlflow_project -  INFO - Created directory at: artifacts
[ 2023-10-24 17:53:26,161 ] 53 common mlflow_project -  INFO - Created directory at: artifacts/data_ingestion
[ 2023-10-24 17:53:27,140 ] 20 3639590181 mlflow_project -  INFO - artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: